In [89]:
import numpy as np 
import matplotlib.pyplot as plt
import matplotlib.patches as patches

In [198]:
# Simulation parameters 
N = 1024
D = 3e-3
dx = D / N
wl = 810e-9
sig = 50e-6
L = 50e-3
focal_length = 20e-3

# X, Y range centered around 0
X = (np.arange(1, N + 1) - (N / 2 + 0.5)) * dx
XX, YY = np.meshgrid(X, X)

# k-space grid
df = 1 / D
freqs = df * np.arange(-N // 2, N // 2)
freq_XXs, freq_YYs = np.meshgrid(freqs, freqs)
light_k = 2 * np.pi / wl
k_xx = freq_XXs * 2 * np.pi
k_yy = freq_YYs * 2 * np.pi
X_crystal_plane = wl * focal_length * freqs
XX_crystal, YY_crystal = np.meshgrid(X_crystal_plane, X_crystal_plane)

k_z_sqr = light_k ** 2 - (k_xx ** 2 + k_yy ** 2)
# Remove all the negative component, as they represent evanescent waves, see Fourier Optics page 58
np.maximum(k_z_sqr, 0, out=k_z_sqr)
k_z = np.sqrt(k_z_sqr)


In [199]:
from pianoq.misc.misc import colorize

def show(E, ax, title, use_colorize=True):
    if use_colorize:
        ax.imshow(colorize(E))
    else:
        I = np.abs(E)**2
        imm = ax.imshow(I)
        ax.figure.colorbar(imm, ax=ax)
    ax.set_title(title)
    
def ft2(E):
    return np.fft.fftshift(np.fft.fft2(np.fft.fftshift(E))) * D

def get_diffuser(M_macro_pixels=128):
    phi_small = np.random.uniform(0, 2*np.pi, size=(M_macro_pixels, M_macro_pixels))
    block_size = N // M_macro_pixels
    diffuser_phi = np.kron(phi_small, np.ones((block_size, block_size)))
    return diffuser_phi

In [200]:
def propagate(diffuser_phi, incidence_angle, pump_waist=20000e-6, debug_mode=False):
    # Assume I began somewhere at the first detector, and reached the diffuser with this size 
    sig = 0.35e-3
    E_at_diffuser = np.exp(-(XX**2 + YY**2)/sig**2)
    # Angle theta so e^(i*k_x*x), with k_x=2pi/wl * sin(theta) 
    angle_factor = (2 * np.pi / wl) * np.sin(incidence_angle)

    E_at_diffuser_angled = E_at_diffuser * np.exp(1j*XX*angle_factor)
    
    E_after_diffuser = E_at_diffuser_angled * np.exp(1j*diffuser_phi)
    
    # Crystal at farfield, so the filter incurred by the pump will not be diagonal in the diffuser plane, 
    # and will result with a finite memory 
    E_at_crystal = ft2(E_after_diffuser)
    pump = np.exp(-(XX_crystal**2 + YY_crystal**2)/pump_waist**2)
    E_crystal_filtered = E_at_crystal * pump
    
    E_before_diffuser_again = ft2(E_crystal_filtered)
    E_after_diffuser_again = E_before_diffuser_again * np.exp(1j*diffuser_phi)
    
    # Not allowed to use the reangle trick, since if pump is very tight - we will completely forget the original angle!  
    # E_reangled = E_after_diffuser_again * np.exp(1j*XX*angle_factor)
    
    E_end = ft2(E_after_diffuser_again)
    
    if debug_mode:
        fig, axes = plt.subplots(2, 4)
        show(E_at_diffuser_angled, axes[0, 0], 'E at diffuser angled')
        show(E_after_diffuser, axes[0, 1], 'E after diffuser')
        show(E_at_crystal, axes[0, 2], 'E at crystal')
        show(pump, axes[0, 3], 'pump')
        show(E_crystal_filtered, axes[1, 0], 'E crystal filtered')
        show(E_before_diffuser_again, axes[1, 1], 'E before diffuser again')
        show(E_after_diffuser_again, axes[1, 2], 'E after diffuser again')
        show(E_end, axes[1, 3], 'End')
        fig.show()
    
    return E_end 
        


In [223]:
def visualize_angles(diffuser_phi, angles, pump_waist):
    E_ends = [] 
    for incidence_angle in angles:
        E_end = propagate(diffuser_phi, incidence_angle, pump_waist)
        E_ends.append(E_end)
    N_angles = len(angles)
    fig, ax = plt.subplots(1, N_angles)
    for i in range(N_angles):
        show(E_ends[i], ax[i], f'E end {angles[i]:.2f}', use_colorize=False)
        ax[i].set_xlim(300, N-300)
        ax[i].set_ylim(300, N-300)
    fig.show()    
    return E_ends

# This is buggy and does not currently work. Perhaps best to just leave the hiccups - it is not worth the trouble...     
def crop_x_subpixel(I, D_area, shift=0.0):
    center = I.shape[0] // 2
    half = D_area // 2
    row_slice = slice(center - half, center + half)
    
    int_shift = int(np.floor(shift))
    frac_shift = shift - int_shift
    
    if shift >= 0:
        col_slice = slice(center - half - int_shift, center + half - int_shift + 1)
        region = I[row_slice, col_slice]
        patch = (1 - frac_shift) * region[:, :-1] + frac_shift * region[:, 1:]
    else:
        col_slice = slice(center - half - int_shift - 1, center + half - int_shift)
        region = I[row_slice, col_slice]
        patch = (1 - frac_shift) * region[:, 1:] + frac_shift * region[:, :-1]

    return patch


def get_PCCs(diffuser_phi, angles, pump_waist, D_area, debug_mode=False):
     
    E_end0 = propagate(diffuser_phi, 0, pump_waist)
    I_end0 = np.abs(E_end0)**2
    ref_I0 = crop_x_subpixel(I_end0, D_area, shift=0)
    
    PCCs = []
    for incidence_angle in angles:
        E_end = propagate(diffuser_phi, incidence_angle, pump_waist)
        I_end = np.abs(E_end)**2
        # Offset due to angle - same speckles but translated 
        fx = np.sin(incidence_angle) / wl   # k_x = k*sin(theta)
        shift = fx / df  # from frequency units to pixel units
        masked_I1 = crop_x_subpixel(I_end, D_area, shift=shift)        
        PCC = np.corrcoef(ref_I0.ravel(), masked_I1.ravel())[0, 1]
        PCCs.append(PCC)
        
    if debug_mode:
        # I might have a bug here with where I add the patch 
        fig, ax = plt.subplots()
        show(I_end, ax, f'I_end0', use_colorize=False)
        rect = patches.Rectangle(
                (N//2 - D_area//2 - round(shift), N//2 - D_area//2),  # Bottom-left corner
                D_area, D_area,  # Width, Height
                linewidth=0.5,  # thin
                edgecolor='white',
                facecolor='none',
                linestyle='dashed'
            )
        ax.add_patch(rect)
        fig.show()
        
    return PCCs

In [153]:
# Debug propagation
diffuser_phi = get_diffuser(M_macro_pixels=128)
_ = propagate(diffuser_phi, incidence_angle=0.03, pump_waist=100e-6, debug_mode=True)

In [178]:
# Visualize experiment 
diffuser_phi = get_diffuser(M_macro_pixels=128)
incidence_angles = np.linspace(0.0, 0.03, 5)
E_ends = visualize_angles(diffuser_phi, incidence_angles, pump_waist=500-6)

In [224]:
# Memory for different pump waists 
incidence_angles = np.linspace(0.0, 0.03, 10)  # 0.03 is a good maximal angle, keeping reasonable resolution for linear phase, and 
                                               # and there is some light close to the optical axis for diffuser with M=128 
pump_waists = np.array([200, 300, 400, 600, 800, 1200])*1e-6
pump_waists = np.array([4200])*1e-6
all_PCCs = []
for pump_waist in pump_waists:
    PCCs = get_PCCs(diffuser_phi, incidence_angles, pump_waist, D_area=50, debug_mode=False)
    all_PCCs.append(PCCs)

fig, ax = plt.subplots()
for i, pump_waist in enumerate(pump_waists):
    angles_deg = (incidence_angles / (2*np.pi)) * 360 
    ax.plot(angles_deg, all_PCCs[i], '.-', label=f'$\sigma_{{p}}$={pump_waist*1e6:.0f}$\mu$m')
    ax.set_ylabel('PCC')
    ax.set_xlabel('angle (deg)')
    ax.legend()
fig.show()